## Apply all Cell-Health Models to Training and Testing Sets

**Gregory Way, 2019**

In [1]:
import os
import pandas as pd
from joblib import load

from scripts.ml_utils import load_train_test, load_models

/Users/gway/miniconda3/envs/cell-health/lib/python3.7/site-packages/dask/array/random.py:27: FutureWarning: dask.array.random.doc_wraps is deprecated and will be removed in a future version
  FutureWarning,


In [2]:
%matplotlib inline

In [3]:
def apply_model(model, feature, train_x, test_x):
    """
    Apply model to training and testing matrix
    """
    pred_train_df = (
        pd.DataFrame(model.predict(train_x), columns=["score"])
        .assign(profiles=train_x.index,
                Metadata_data_type="train",
                model=feature)
    )
    pred_test_df = (
        pd.DataFrame(model.predict(test_x), columns=["score"])
        .assign(profiles=test_x.index,
                Metadata_data_type="test",
                model=feature)
    )

    pred_df = pd.concat([pred_train_df, pred_test_df]).reset_index(drop=True)
    return pred_df

def sample_squared_error(scores, y):
    """
    Calculate the squared error per sample depending on model scores
    """
    metadata_cols = [x for x in scores.columns if x.startswith("Metadata_")]
    scores_values = scores.drop(metadata_cols, axis="columns")
    
    all_squared_error = {}
    for cell_health_feature in scores_values.columns:
        y_subset_df = y.loc[:, cell_health_feature].dropna().T
        scores_subset = scores_values.loc[:, cell_health_feature].reindex(y_subset_df.index).T

        squared_error = (y_subset_df - scores_subset) ** 2
        all_squared_error[cell_health_feature] = squared_error
    
    return pd.DataFrame(all_squared_error).reindex(scores.index)

## 1) Load Models and Model Coefficients

For real data and shuffled model data.

In [4]:
consensus = "modz"

In [5]:
model_dict, model_coef = load_models(consensus=consensus)
shuffle_model_dict, shuffle_model_coef = load_models(shuffle=True, consensus=consensus)

In [6]:
# Load Metadata Mapping File
data_dir = os.path.join("..", "1.generate-profiles", "data")
file = os.path.join(data_dir, "profile_id_metadata_mapping.tsv")
metadata_df = pd.read_csv(file, sep='\t')

metadata_df.head()

,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name
0,profile_0,A549,AKT1-1
1,profile_1,A549,AKT1-2
2,profile_2,A549,ARID1B-1
3,profile_3,A549,ARID1B-2
4,profile_4,A549,ATF4-1


## 2) Load Training and Testing Data

In [7]:
x_train_df, x_test_df, y_train_df, y_test_df = load_train_test(drop_metadata=True, consensus=consensus)

## 3) Output Model Coefficients

In [8]:
# Extract all model coefficients and output to file
coef_df = pd.DataFrame(model_coef)
coef_df.index = x_test_df.columns
coef_df.index.name = "features"

file = os.path.join("results",
                    "all_model_coefficients_{}.tsv".format(consensus))
coef_df.to_csv(file, sep='\t', index=True)

print(coef_df.shape)
coef_df.head(2)

(949, 70)


,cell_health_modz_target_cc_late_mitosis_n_spots_h2ax_per_nucleus_area_mean,cell_health_modz_target_cc_all_nucleus_roundness_mean,cell_health_modz_target_cc_polyploid_n_objects,cell_health_modz_target_cc_g1_n_objects,cell_health_modz_target_cc_mitosis_n_objects,cell_health_modz_target_vb_num_live_cells,cell_health_modz_target_vb_percent_live,cell_health_modz_target_vb_live_cell_roundness,cell_health_modz_target_cc_all_large_notround_polynuclear_mean,cell_health_modz_target_cc_all_high_h2ax,...,cell_health_modz_target_cc_s_intensity_nucleus_area_mean,cell_health_modz_target_cc_g2_high_h2ax,cell_health_modz_target_cc_late_mitosis_n_spots_h2ax_mean,cell_health_modz_target_cc_cc_high_h2ax,cell_health_modz_target_vb_percent_dead_only,cell_health_modz_target_cc_s_high_h2ax,cell_health_modz_target_cc_cc_n_spots_h2ax_per_nucleus_area_mean,cell_health_modz_target_cc_s_n_spots_h2ax_per_nucleus_area_mean,cell_health_modz_target_cc_g1_plus_g2_count,cell_health_modz_target_vb_live_cell_width_length
features,,,,,,,,,,,,,,,,,,,,,
Cells_AreaShape_Center_Y,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,...,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,-0.0
Cells_AreaShape_Compactness,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0


In [9]:
# Extract all model coefficients and output to file
shuffle_coef_df = pd.DataFrame(shuffle_model_coef)
shuffle_coef_df.index = x_test_df.columns
shuffle_coef_df.index.name = "features"

file = os.path.join("results",
                    "all_model_coefficients_shuffled_{}.tsv".format(consensus))
shuffle_coef_df.to_csv(file, sep='\t', index=True)

print(shuffle_coef_df.shape)
shuffle_coef_df.head(2)

(949, 70)


,cell_health_modz_target_cc_all_n_objects,cell_health_modz_target_cc_g1_high_h2ax,cell_health_modz_target_vb_num_live_cells,cell_health_modz_target_cc_all_high_h2ax,cell_health_modz_target_cc_g1_n_objects,cell_health_modz_target_cc_g1_n_spots_h2ax_per_nucleus_area_mean,cell_health_modz_target_cc_late_mitosis_n_spots_h2ax_mean,cell_health_modz_target_cc_cc_n_spots_h2ax_per_nucleus_area_mean,cell_health_modz_target_cc_cc_g1,cell_health_modz_target_cc_polynuclear_n_spots_h2ax_per_nucleus_area_mean,...,cell_health_modz_target_cc_all_n_spots_h2ax_per_nucleus_area_mean,cell_health_modz_target_cc_s_intensity_nucleus_area_sum,cell_health_modz_target_cc_early_mitosis_n_spots_h2ax_mean,cell_health_modz_target_cc_g1_n_spots_h2ax_mean,cell_health_modz_target_cc_cc_mitosis,cell_health_modz_target_cc_polynuclear_n_objects,cell_health_modz_target_cc_polynuclear_high_h2ax,cell_health_modz_target_cc_s_intensity_nucleus_area_mean,cell_health_modz_target_cc_early_mitosis_n_objects,cell_health_modz_target_cc_early_mitosis_high_h2ax
features,,,,,,,,,,,,,,,,,,,,,
Cells_AreaShape_Center_Y,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,...,0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,0.00000,-0.0
Cells_AreaShape_Compactness,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,0.00128,-0.0


## 4) Apply all models

For real and shuffled data.

In [10]:
all_scores = []
all_shuffle_scores = []
for cell_health_feature in model_dict.keys():
    # Apply Real Model Classifiers
    model_clf = model_dict[cell_health_feature]
    pred_df = apply_model(model=model_clf,
                          feature=cell_health_feature,
                          train_x=x_train_df,
                          test_x=x_test_df)
    all_scores.append(pred_df)
    
    # Apply Shuffled Model Classifiers
    shuffle_model_clf = shuffle_model_dict[cell_health_feature]
    shuffle_pred_df = apply_model(model=shuffle_model_clf,
                                  feature=cell_health_feature,
                                  train_x=x_train_df,
                                  test_x=x_test_df)
    all_shuffle_scores.append(shuffle_pred_df)

## 5) Concatenate scores with Metadata

In [11]:
# Concatenate real data scores
all_scores = (
    pd.concat(all_scores)
    .reset_index(drop=True)
    .pivot_table(index=["profiles", "Metadata_data_type"],
                 columns="model",
                 values="score")
    .reset_index()
)

all_scores = (
    metadata_df.merge(
        all_scores,
        left_on="Metadata_profile_id",
        right_on="profiles"
    )
    .drop("profiles", axis="columns")
)

all_scores.index = all_scores.Metadata_profile_id
all_scores = all_scores.drop("Metadata_profile_id", axis="columns")

# Remove prefix of variable columns
strip_text = "cell_health_{}_target_".format(consensus)
all_scores.columns = [x.replace(strip_text, "") for x in all_scores.columns]

# Output file
file = os.path.join(
    "results", "all_model_predictions_{}.tsv".format(consensus)
)
all_scores.to_csv(file, sep='\t', index=True)

print(all_scores.shape)
all_scores.head(2)

(357, 73)


,Metadata_cell_line,Metadata_pert_name,Metadata_data_type,cc_all_high_h2ax,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_h2ax_mean,cc_all_n_spots_h2ax_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_early_apoptosis,vb_percent_late_apoptosis,vb_percent_live,vb_ros_back_mean,vb_ros_mean
Metadata_profile_id,,,,,,,,,,,,,,,,,,,,,
profile_0,A549,AKT1-1,train,0.219139,0.347752,-0.014601,0.151728,0.273698,0.216029,-0.090430,...,0.021872,0.144581,0.306686,0.150832,0.201712,0.150316,0.058777,-0.151460,0.281601,0.317835
profile_1,A549,AKT1-2,train,0.180354,0.313098,0.242201,0.215245,0.109491,0.138052,-0.114391,...,0.036097,0.085218,0.281135,0.190021,0.250567,0.110907,0.156190,-0.190896,0.081968,0.392189


In [12]:
# Concatenate shuffled data scores
all_shuffle_scores = (
    pd.concat(all_shuffle_scores)
    .reset_index(drop=True)
    .pivot_table(index=["profiles", "Metadata_data_type"],
                 columns="model",
                 values="score")
    .reset_index()
)

all_shuffle_scores = (
    metadata_df.merge(all_shuffle_scores,
                      left_on="Metadata_profile_id",
                      right_on="profiles")
    .drop("profiles", axis="columns")
)

all_shuffle_scores.index = all_shuffle_scores.Metadata_profile_id
all_shuffle_scores = all_shuffle_scores.drop("Metadata_profile_id", axis="columns")

# Remove prefix of variable columns
strip_text = "cell_health_{}_target_".format(consensus)
all_shuffle_scores.columns = [x.replace(strip_text, "") for x in all_shuffle_scores.columns]

# Output file
file = os.path.join(
    "results", "all_model_predictions_shuffled_{}.tsv".format(consensus)
)
all_shuffle_scores.to_csv(file, sep='\t', index=True)

print(all_shuffle_scores.shape)
all_shuffle_scores.head(2)

(357, 73)


,Metadata_cell_line,Metadata_pert_name,Metadata_data_type,cc_all_high_h2ax,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_h2ax_mean,cc_all_n_spots_h2ax_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_early_apoptosis,vb_percent_late_apoptosis,vb_percent_live,vb_ros_back_mean,vb_ros_mean
Metadata_profile_id,,,,,,,,,,,,,,,,,,,,,
profile_0,A549,AKT1-1,train,0.283408,0.302838,0.261533,-0.052079,0.242281,0.276496,0.176105,...,-0.263365,0.222611,0.222697,0.179944,0.267044,0.152208,0.231602,-0.327383,-0.007581,0.027778
profile_1,A549,AKT1-2,train,0.301200,0.332812,0.261533,-0.070843,0.228718,0.276496,0.138127,...,-0.263365,0.223070,0.236935,0.264971,0.267044,0.152384,0.231602,-0.273873,-0.007581,0.027778


## 6) Calculate the Squared Error of Individual Samples

For real and shuffled data

In [13]:
y_df = pd.concat([y_train_df, y_test_df]).reindex(all_scores.index)

print(y_df.shape)
y_df.head(2)

(357, 70)


,cc_all_high_h2ax,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_h2ax_mean,cc_all_n_spots_h2ax_per_nucleus_area_mean,cc_all_nucleus_area_mean,cc_all_nucleus_roundness_mean,cc_cc_early_mitosis,cc_cc_g1,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_early_apoptosis,vb_percent_late_apoptosis,vb_percent_live,vb_ros_back_mean,vb_ros_mean
Metadata_profile_id,,,,,,,,,,,,,,,,,,,,,
profile_0,-0.005795,0.580351,0.013975,0.381958,0.150696,0.162511,-0.167603,0.040322,0.375966,0.043915,...,0.438339,0.059414,-0.065050,-0.020236,-0.007970,0.082424,0.000000,0.020263,0.408214,0.654575
profile_1,0.050169,1.277730,0.241808,0.577422,0.220829,0.366989,-0.278044,-0.182571,0.736046,-1.023968,...,0.067568,0.256141,0.575026,0.225091,0.220461,0.132834,0.386327,-0.224965,0.284962,0.567898


In [14]:
all_score_error = sample_squared_error(scores=all_scores, y=y_df)

all_score_error = (
    metadata_df.merge(
        all_score_error,
        left_on="Metadata_profile_id",
        right_index=True
    )
)

# Output file
file = os.path.join(
    "results", "all_model_sample_squared_error_{}.tsv".format(consensus)
)
all_score_error.to_csv(file, sep='\t', index=False)

print(all_score_error.shape)
all_score_error.head(2)

(357, 73)


,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name,cc_all_high_h2ax,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_h2ax_mean,cc_all_n_spots_h2ax_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_early_apoptosis,vb_percent_late_apoptosis,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,profile_0,A549,AKT1-1,0.050595,0.054102,8.165820e-04,0.053006,0.015129,0.002864,0.005956,...,0.173445,0.007253,0.138188,0.029265,0.043966,0.004609,0.003455,0.029489,0.016031,0.113393
1,profile_1,A549,AKT1-2,0.016948,0.930516,1.545028e-07,0.131172,0.012396,0.052412,0.026782,...,0.000990,0.029215,0.086372,0.001230,0.000906,0.000481,0.052963,0.001161,0.041206,0.030874


In [15]:
all_shuffle_score_error = sample_squared_error(scores=all_shuffle_scores, y=y_df)

all_shuffle_score_error = (
    metadata_df.merge(
        all_shuffle_score_error,
        left_on="Metadata_profile_id",
        right_index=True
    )
)

# Output file
file = os.path.join(
    "results", "all_model_sample_squared_error_shuffled_{}.tsv".format(consensus)
)
all_shuffle_score_error.to_csv(file, sep='\t', index=False)

print(all_shuffle_score_error.shape)
all_shuffle_score_error.head()

(357, 73)


,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name,cc_all_high_h2ax,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_h2ax_mean,cc_all_n_spots_h2ax_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_early_apoptosis,vb_percent_late_apoptosis,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,profile_0,A549,AKT1-1,0.083638,0.077014,0.061285,0.188387,0.008388,0.012993,0.118136,...,0.492389,0.026633,0.082798,0.040072,0.075633,0.004870,0.053640,0.120858,0.172885,0.392873
1,profile_1,A549,AKT1-2,0.063016,0.892871,0.000389,0.420247,0.000062,0.008189,0.173199,...,0.109516,0.001094,0.114305,0.001590,0.002170,0.000382,0.023940,0.002392,0.085581,0.291729
2,profile_2,A549,ARID1B-1,0.001842,0.946663,0.009291,0.059281,0.009616,0.056439,0.194469,...,0.132299,0.005449,0.047055,0.031011,0.048113,0.009442,0.002667,0.037270,0.126868,0.054623
3,profile_3,A549,ARID1B-2,0.030760,0.015668,0.123070,0.000056,0.388985,0.332670,0.079861,...,0.299392,0.006259,0.033898,0.100248,0.124399,0.000245,0.014339,0.151583,0.042381,0.036387
4,profile_4,A549,ATF4-1,18.138688,0.056188,9.137624,6.662628,7.461677,7.780065,6.031018,...,4.712250,0.010302,0.003566,0.007877,0.026305,0.032464,0.001500,0.225108,0.645918,0.014468
